In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.ensemble import RandomForestRegressor as RFR
from  sklearn.utils import resample

In [2]:
data = pd.read_csv('insurance.csv', sep=',')
data_train = data.sample(frac=0.9)
data_validate = data.loc[~data.index.isin(data_train.index)]
data_train

,age,sex,bmi,children,smoker,region,charges
1221,40,male,24.970,2,no,southeast,6593.50830
969,39,female,34.320,5,no,southeast,8596.82780
377,24,male,40.150,0,yes,southeast,38126.24650
1047,22,male,52.580,1,yes,southeast,44501.39820
587,34,female,30.210,1,yes,northwest,43943.87610
...,...,...,...,...,...,...,...
207,35,male,27.740,2,yes,northeast,20984.09360
595,46,female,33.725,1,no,northeast,8823.98575
1140,50,male,37.070,1,no,southeast,9048.02730
325,40,male,34.105,1,no,northeast,6600.20595


# **L1 CONFIDENCE INTERVAL 95**

**CREATE COLUMN TRANSFORMER**

In [3]:
col_trans_l1 = ColumnTransformer([
    'scaller', StandardScaler(), ('age', 'bmi', 'children'),
    'hot_encode', OneHotEncoder(handle_unknown='ignore', drop='first'), ('sex', 'smoker', 'region'),
], remainder='passthrough')

**CREATE PIPE**

In [4]:
pipe_l1 = Pipeline([
    ('col_trans_l1', col_trans_l1), 
    ('poly', PolynomialFeatures(degree=2, include_bias=False)), 
    ('lasso', Lasso(alpha=74.58219397993311))
]), 

**GENERATE SAMPLES**

In [5]:
btstrap = resample(data_train)

In [6]:
btstrap

,age,sex,bmi,children,smoker,region,charges
253,27,male,30.300,3,no,southwest,4260.74400
765,57,female,31.825,0,no,northwest,11842.62375
555,28,male,23.800,2,no,southwest,3847.67400
893,47,male,38.940,2,yes,southeast,44202.65360
813,28,male,22.515,2,no,northeast,4428.88785
...,...,...,...,...,...,...,...
1063,36,male,28.595,3,no,northwest,6548.19505
151,48,male,29.700,0,no,southeast,7789.63500
762,33,male,27.100,1,yes,southwest,19040.87600
636,19,female,24.510,1,no,northwest,2709.11190
